# Transfer Learning with Xception

In [1]:
# download dataset
# !gdown "https://drive.google.com/uc?id=1dIxJQ8G5GlxV5tb3meBk88QC6gk-rpzC" -O dataset.zip

In [2]:
# download test dataset
# !gdown "https://drive.google.com/uc?id=15VGocJ8uCQaagg5_nyDIoenGOFfeVSxR" -O test.zip

In [3]:
# !rm -r dataset
# !rm -r test
# !unzip dataset.zip -d dataset
# !unzip test.zip -d test

## Import Libraries

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# avoid error
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Model Hypertunning
# !pip install -q keras_tuner
import keras_tuner as kt

## Data Preprocessing

In [5]:
train_data_dir = 'Dataset/train & validation'
test_data_dir = 'Dataset/test'

### Preview Data

In [6]:
# print("Sample day image:")
# plt.imshow(load_img(f"{os.path.join(day_dir, os.listdir(day_dir)[0])}"))
# plt.show()

# print("\nSample night image:")
# plt.imshow(load_img(f"{os.path.join(night_dir, os.listdir(night_dir)[0])}"))
# plt.show()

### Settings

In [7]:
# Get all class names and count the number of classes
class_names = os.listdir(train_data_dir)
n_classes = len(class_names)

# Set some constants for the dataset
BATCH_SIZE = 32 # Number of samples in each batch during training
IMAGE_SIZE = 224 # Size of the image
# AUTOTUNE = tf.data.AUTOTUNE # Set to optimize the buffer size automatically
LEARNING_RATE = 1e-3 # Learning rate for the optimizer used during model training

In [8]:
# Set the random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

### Image Augmentation

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                #    rotation_range=40,
                                #    width_shift_range=0.2,
                                #    height_shift_range=0.2,
                                #    shear_range=0.2,
                                #    zoom_range=0.2,
                                #    horizontal_flip=True,
                                #    fill_mode='nearest'
                                validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')



Found 0 images belonging to 0 classes.


In [10]:
# Set up the data generator for training and validation
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory(train_data_dir,
                                                         target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                         batch_size=BATCH_SIZE,
                                                         class_mode='categorical',
                                                         subset='validation')

Found 4023 images belonging to 13 classes.
Found 1000 images belonging to 13 classes.


## Modelling and Tuning

In [11]:
# # Add your custom layers on top of the pre-trained model
xception = tf.keras.applications.Xception(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                                          weights='imagenet',
                                          include_top=False)

# Freeze the model weights
xception.trainable = True

# model = Sequential([
#     xception,
#     GlobalAveragePooling2D(),
#     Dropout(0.5),
#     Dense(n_classes, activation='softmax')
# ])

# # Compile the model
# model.compile(
#     # loss='sparse_categorical_crossentropy',
#     loss='categorical_crossentropy',
#     optimizer=Adam(learning_rate=LEARNING_RATE),
#     metrics=['accuracy']
# )

# model.fit(
#     train_generator,
#     validation_data=validation_generator, 
#     epochs=50, 
#     callbacks=[
#         EarlyStopping(patience=3, restore_best_weights=True),
#         ModelCheckpoint("XceptionModel.h5", save_best_only=True)
#     ],
#     batch_size=BATCH_SIZE
# )

83683744/83683744 [==============================] - 203s 2us/step


In [12]:
# # best_test_loss, best_test_acc = best_xception.evaluate(X_test, y_test)
# # print(f"Test Loss after Tunig     : {best_test_loss} | {xtest_loss}")
# # print(f"Test Accuracy after Tunig : {best_test_acc}  | {xtest_acc}")

# xtest_loss, xtest_acc = model.evaluate(test_generator)
# print(f"Xception Baseline Testing Loss     : {xtest_loss}.")
# print(f"Xception Baseline Testing Accuracy : {xtest_acc}.")

In [13]:
def build_model(hp, n_classes=13):
    
    # Define all hyperparms
    n_layers = hp.Choice('n_layers', [0, 2, 4])
    dropout_rate = hp.Choice('rate', [0.2, 0.4, 0.5, 0.7])
    n_units = hp.Choice('units', [64, 128, 256, 512])
    
    # Mode architecture
    model = Sequential([
        xception,
        GlobalAveragePooling2D(),
    ])
    
    # Add hidden/top layers 
    for _ in range(n_layers):
        model.add(Dense(n_units, activation='relu', kernel_initializer='he_normal'))
    
    # Add Dropout Layer
    model.add(Dropout(dropout_rate))
    
    # Output Layer
    model.add(Dense(n_classes, activation='softmax'))
    
    # Compile the model
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer = Adam(LEARNING_RATE),
        metrics = ['accuracy']
    )
    
    # Return model
    return model

In [14]:
# Initialize Random Searcher
random_searcher = kt.RandomSearch(
    hypermodel=build_model, 
    objective='val_loss', 
    max_trials=10, 
    seed=42, 
    project_name="XceptionSearch", 
    # loss='sparse_categorical_crossentropy'
    loss='categorical_crossentropy'
)

# Start Searching
search = random_searcher.search(
    train_generator,
    validation_data=validation_generator,
    epochs = 10,
    batch_size = BATCH_SIZE
)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |2                 |n_layers
0.2               |0.2               |rate
128               |128               |units

Epoch 1/10


c:\Users\danie\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


126/126 [==============================] - 790s 6s/step - loss: 0.8223 - val_loss: 3.8184
Epoch 2/10
126/126 [==============================] - 770s 6s/step - loss: 0.4567 - val_loss: 1.5514
Epoch 3/10
126/126 [==============================] - 773s 6s/step - loss: 0.3654 - val_loss: 2.6150
Epoch 4/10
126/126 [==============================] - 811s 6s/step - loss: 0.3195 - val_loss: 0.7968
Epoch 5/10
126/126 [==============================] - 867s 7s/step - loss: 0.2723 - val_loss: 1.0445
Epoch 6/10
 56/126 [============>.................] - ETA: 7:18 - loss: 0.1972

KeyboardInterrupt: 

In [ ]:
# Collect the best model Xception Model Architecture obtained by Random Searcher
best_xception = build_model(random_searcher.get_best_hyperparameters(num_trials=1)[0])

# Model Architecture
best_xception.summary()

# Compile Model
best_xception.compile(
    # loss='sparse_categorical_crossentropy',
    loss='categorical_crossentropy',
    optimizer=Adam(LEARNING_RATE*0.1),
    metrics=['accuracy']
)

# Model Training
best_xception_history = best_xception.fit(
    train_generator,
    validation_data=validation_generator,
    epochs = 50,
    batch_size = BATCH_SIZE*2,
    callbacks = [
        EarlyStopping(patience=2, restore_best_weights=True),
        ModelCheckpoint("BestXception.h5", save_best_only=True)
    ]
)

loss, accuracy = best_xception.evaluate(test_generator)
print(f"Test Loss after Tunig     : {loss}")
print(f"Test Loss after Tunig     : {accuracy}")
# print(f"Test Loss after Tunig     : {loss} | {xtest_loss}")
# print(f"Test Accuracy after Tunig : {accuracy}  | {xtest_acc}")

### Pre-Trained Model Load

In [ ]:
#  Load model 
best_xception = tf.keras.models.load_model('/BestXception.h5', compile=False)
best_xception.summary()

## Model Evaluation

In [ ]:
def plot_graghs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel('Epochs')
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

### Accuracy

In [ ]:
plot_graghs(best_xception_history, 'accuracy')

### Loss

In [ ]:
plot_graghs(best_xception_history, 'loss')

## Model Predictions

In [ ]:
# import numpy as np
# from keras.utils import load_img, img_to_array

# label = ['battery', 'cable', 'e_kettle', 'keyboard', 'laptop', 'light_bulb', 'monitor', 'mouse', 'pcb', 'phone', 'printer', 'rice_cooker', 'tv']
# for fn in os.listdir('test'):
#   path = './test' + fn
#   img = load_img(path, target_size=(150, 150))
#   x = img_to_array(img)
#   x /= 255
#   x = np.expand_dims(x, axis=0)

#   images = np.vstack([x])
#   classes = model.predict(images, batch_size=10)
#   print(fn)
#   for i, l in zip(classes[0], label):
#     print("{} : {:.2%}".format(l, i))

## Save the Model

In [ ]:
# Save the model
# model.save('e_waste_classifier.h5')